### Обучение матриц проекции

Реализуем процесс обучения матриц проекции, который был предложен на SemEval CRIM. Для этого сначала требуется загрузить какие-нибудь эмбеддинги для слов и подобрать то, каких именно кандидатов требуется ранжировать при выдаче гиперонимов.

Начнём с простой стратегии. Возьмём обученную модель FastText и загрузим эмбеддинги из неё. После этого будем для многословных термов усреднять эмбеддинги.

In [1]:
import sys
import json
from os.path import join
import os
sys.path.append("../")
import fasttext as ft
from thesaurus_parsing.thesaurus_parser import ThesaurusParser
from collections import Counter
from tqdm import tqdm_notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

%matplotlib inline

In [2]:
deeppavlov_embeddings = ft.load_model('../data/models/fasttext_deeppavlov.bin')

In [3]:
len(deeppavlov_embeddings.get_words())

1572343

Мы видим, что здесь есть векторы для полутора миллионов слов. Конечно же, использовать их все, как кандидаты в гиперонимы, было бы хорошо. Но тем не менее, поскольку метод в основном похож на kNN, это будет очень долго.

Вследствие этого, необходимо, кроме сущностей тезауруса, оставить лишь некоторый топ в качестве кандидатов в гиперонимы. Этот топ можно подобрать по tf-idf. Но для начала надо построить векторы для сущностей из тезауруса.

In [4]:
thesaurus = ThesaurusParser("../data/RuThes", need_closure=False)

In [5]:
vocab_embeddings = dict()

In [6]:
for _, entry_dict in thesaurus.text_entries.items():
    lemma = entry_dict['lemma']
    vocab_embeddings[lemma] = deeppavlov_embeddings.get_sentence_vector(lemma)

In [7]:
len(vocab_embeddings)

110176

Давайте пройдём по всем текстам, которые загрузились на данный момент, для слов, которые есть в словаре, посчитаем частоту слов

In [8]:
DIR_PATH = "/home/loginov-ra/MIPT/HypernymyDetection/data/Lenta/texts_tagged_processed_tree"
file_list = os.listdir(DIR_PATH)
file_list = [join(DIR_PATH, filename) for filename in file_list]

In [9]:
word_ctr = Counter()
no_deeppavlov = 0

for filename in tqdm(file_list):
    with open(filename, encoding='utf-8') as sentences_file:
        sentences = json.load(sentences_file)
        for sent in sentences:
            if 'deeppavlov' not in sent:
                no_deeppavlov += 1
                continue
            
            multitokens, _ = sent['multi']
            for t in multitokens:
                word_ctr[t] += 1

In [10]:
print(no_deeppavlov)

883


In [11]:
word_ctr.most_common(n=28)

[(',', 521733),
 ('.', 445009),
 ('в', 307206),
 ('"', 299914),
 ('на', 125625),
 ('и', 123598),
 ('-', 111278),
 ('с', 77467),
 ('что', 72277),
 ('быть', 70839),
 ('по', 70425),
 ('год', 57392),
 ('о', 52364),
 (')', 49128),
 ('(', 48432),
 ('не', 47363),
 ('который', 42268),
 ('он', 40684),
 (':', 40304),
 ('это', 37290),
 ('из', 37212),
 ('тот', 32196),
 ('за', 28677),
 ('как', 28572),
 ('один', 27042),
 ('--', 26014),
 ('к', 23407),
 ('сообщать', 22883)]

Видим, что для первых $27$ слов нет необходимости искать гиперонимы, для остальных уже может быть. Поэтому возьмёи пока первые $100000$ слов для работы с ними, посчитаем их эмбеддинги и добавим в словарь.

In [12]:
additional_words = word_ctr.most_common(n=100000)[27:]

In [13]:
for word, _ in additional_words:
    vocab_embeddings[word] = deeppavlov_embeddings.get_word_vector(word)

Удалим модель за ненадобностью

In [14]:
del deeppavlov_embeddings

In [15]:
len(vocab_embeddings)

172251

Видим, что было пересечение, и добавилась где-то 61000 слов
_________________

**Определение модели**

Определим модель, в которой будет 5 матриц проекции и логистическая регрессия на косинусных расстояниях до проекций.

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
device

'cpu'

In [49]:
class CRIMModel(nn.Module):
    def __init__(self, n_matrices=5, embedding_dim=300, init_sigma=0.01):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.n_matrices = n_matrices
        self.init_sigma = init_sigma
        
        matrix_shape = (n_matrices, 1, embedding_dim, embedding_dim)
        self.matrices = torch.FloatTensor(size=matrix_shape)
        self.prob_layer = nn.Linear(in_features=n_matrices, out_features=1)
        
        for i in range(n_matrices):
            eye_tensor = torch.FloatTensor(size=(embedding_dim, embedding_dim), device=device)
            noise_tensor = torch.FloatTensor(size=(embedding_dim, embedding_dim), device=device)
            torch.nn.init.eye_(eye_tensor)
            torch.nn.init.normal_(noise_tensor, std=init_sigma)
            self.matrices[i][0] = eye_tensor + noise_tensor
            
        torch.nn.init.normal_(self.prob_layer.weight, std=0.1)
        self.matrices = nn.Parameter(self.matrices.requires_grad_())
        
    def forward(self, input_dict):
        candidate = input_dict['candidate']
        candidate_batch = candidate.shape[0]
        candidate = candidate.view((candidate_batch, 1, self.embedding_dim))
        batch = input_dict['batch'].unsqueeze(-1)
        batch_size = batch.shape[0]
        projections = torch.matmul(self.matrices, batch).permute(1, 0, 2, 3).squeeze(-1)
        similarities = F.cosine_similarity(projections, candidate, dim=-1)
        logits = self.prob_layer(similarities)
        probas = torch.sigmoid(logits)
        return probas

In [37]:
model = CRIMModel()

In [38]:
args = {
    'batch': torch.randn(64, 300),
    'candidate': torch.randn(1, 300)
}

In [39]:
probas = model(args)

In [40]:
optimizer = Adam(model.parameters(), lr=1)

In [41]:
while True:
    probas = model(args)
    loss = probas.sum()
    print('Loss:', loss.detach().numpy())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

Loss: 35.45905
Loss: 9.963653
Loss: 0.77415305
Loss: 0.17260851
Loss: 0.056600925
Loss: 0.02204978
Loss: 0.0098106
Loss: 0.0048678904
Loss: 0.0026406413
Loss: 0.0015418465
Loss: 0.0009574688
Loss: 0.00062648166
Loss: 0.000428732
Loss: 0.00030505323
Loss: 0.00022457582
Loss: 0.00017036783
Loss: 0.00013272851
Loss: 0.00010588361
Loss: 8.6275795e-05
Loss: 7.164689e-05
Loss: 6.0523584e-05
Loss: 5.1920768e-05
Loss: 4.5164634e-05
Loss: 3.9785413e-05
Loss: 3.5448786e-05
Loss: 3.191338e-05
Loss: 2.9001752e-05
Loss: 2.6581678e-05
Loss: 2.4553427e-05
Loss: 2.28407e-05
Loss: 2.1384463e-05
Loss: 2.0138586e-05
Loss: 1.9066734e-05
Loss: 1.813982e-05
Loss: 1.7334512e-05
Loss: 1.6631917e-05
Loss: 1.601655e-05
Loss: 1.5475733e-05
Loss: 1.4998867e-05
Loss: 1.4577208e-05
Loss: 1.4203378e-05


KeyboardInterrupt: 

Во всяком случае, на текущий момент возможно переобучить модель под нужные значения
_________________

**Цикл обучения модели**

Для того, чтобы обучить модель, надо сделать следующие шаги:

* Добавить пары корректных гипонимов-гиперонимов
* Для каждого положительного добавить несколько отрицательных (пока просто случайные слова)
* Добавить вектор правильных ответов

Сделаем из этого `torch.Dataset`

In [42]:
class HypernymQueriesDataset(Dataset):
    def load_train_items(self, thesaurus, vocab):
        self.train_items = []
        for hyponym, hypernyms in tqdm(thesaurus.hypernyms_dict.items()):
            for hypernym in hypernyms:
                self.train_items.append([hyponym, hypernym, True])
            
            negative_examples = np.random.choice(list(vocab.keys()), size=self.n_negative * len(hypernyms))
            for negative in negative_examples:
                self.train_items.append([hyponym, negative, False])
                
            if self.max_pairs is not None and len(self.train_items) > self.max_pairs:
                break
    
    def __init__(self, thesaurus, vocab, n_negative=3, max_pairs=None):
        self.n_negative = n_negative
        self.thesaurus = thesaurus
        self.vocab = vocab
        self.max_pairs = max_pairs
        
        self.load_train_items(thesaurus, vocab)
        
    def __len__(self):
        return len(self.train_items)
    
    def __getitem__(self, idx):
        hyponym, hypernym, label = self.train_items[idx]
        return (self.vocab[hyponym], self.vocab[hypernym], label)

In [43]:
dataset = HypernymQueriesDataset(thesaurus, vocab_embeddings, max_pairs=1000)

In [44]:
model = CRIMModel(n_matrices=24)
optimizer = Adam(model.parameters(), lr=3e-4)
n_epochs = 1
batch_size = 64
plot_frequency = 40

In [45]:
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [46]:
loss_history = []

for epoch in range(n_epochs):
    print('Epoch {}. Started training.'.format(epoch + 1))

    for it, batch in tqdm(enumerate(data_loader), total=len(dataset) / batch_size):
        hyponyms, candidates, labels = batch

        model_batch = {
            'batch': hyponyms,
            'candidate': candidates
        }

        labels = labels.float()
        probas = model(model_batch).squeeze()
        probas = torch.clamp(probas, 1e-5, 1. - 1e-5)
        loss = torch.sum(labels * torch.log(probas) + (1. - labels) * torch.log(1. - probas))
        loss_history.append(loss.detach().numpy())
        
        if len(loss_history) % plot_frequency == 0:
            print(loss)
            clear_output(wait=True)
            plt.figure(figsize=(12, 7))
            plt.title('Loss history', fontsize=18)
            plt.xlabel('Iteration', fontsize=15)
            plt.ylabel('CE loss', fontsize=15)
            plt.plot(np.array(loss_history))
            plt.show()
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

Epoch 1. Started training.


In [51]:
torch.save(model.state_dict(), '../data/models/projection_model.bin')